In [ ]:
import requests
import json
import time

from google.colab import auth
import gspread
from google.auth import default
import pandas as pd

In [ ]:
def get_refresh_token():
  parms = {"refreshtoken":"hRv3j2VHiixfpBRRWi5SA6zx9acCAIB1rXY8ZcanU9nETNxj3o52giWbYVUO8hCU"}
  token_url = 'https://api.chartmetric.com/api/token'
  token_res = requests.post(token_url, data=parms)
  refresh_token = token_res.json()['token']
  return refresh_token

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheet = gc.open('Youtube-Data-CE')
ws = worksheet.worksheet("Raw_Data_V2.0")
rows = ws.get_all_values()

df_mod = pd.DataFrame.from_records(rows[1:], columns=rows[0])

In [ ]:
# Function to get track_id:
def get_track_id(id_list, refresh_token):
    stats_result = {'artist_id':[],
                    'track_id': []
                    }

    for id in id_list:
     headers= {'Authorization' : 'Bearer '+refresh_token}
     stats_url = 'https://api.chartmetric.com/api/artist/'+id+'/tracks'
     stats_res = requests.get(stats_url, headers=headers)
     print(id, end=" ")
     time.sleep(1)
     if 'obj' in stats_res.json():
      for i in range(len(stats_res.json()['obj'])):
        stats_result['artist_id'].append(id)
        stats_result['track_id'].append(stats_res.json()['obj'][i]['cm_track'])
    return stats_result

In [ ]:
refresh_token = get_refresh_token()

df1 = pd.DataFrame(get_track_id(list(df_mod['artist_id'].iloc[3501:3646]), refresh_token))

295 1004724 12650 89828 792 745 4127288 501 3379 725162 148442 425493 213785 206511 135946 1359481 207062 229871 1328287 126746 4394050 1862 208843 149442 193040 100212 361066 426120 209644 921 468870 9439888 208826 452167 3056 213806 3773130 215035 182284 34839 1465954 2952 4631 3541059 207071 208688 4345579 1051989 209060 5466 209450 1339661 206452 79358 3529061 3744848 2613 879618 214057 208765 810752 212556 3953 2457 105886 208143 729170 417377 1063 25206 4814 61569 165167 214758 1259 207501 1087 3884810 72951 783447 58466 176258 3680068 680 6123621 84842 2598 3489284 828185 51998 4108580 1205393 9858365 774150 526024 493760 1204238 668 53964 14 600674 273743 873626 3788825 3315521 712345 210053 1065 2872 784076 487991 726202 1576233 443 326834 1849 209467 179842 1481401 207562 307408 748366 966 743727 2468 723667 389423 9589911 348085 129837 3711377 206545 3723 206356 86956 1364091 9074013 517256 1016721 155921 3737128 18895 890794 54 257803 

In [ ]:
worksheet = gc.open('Youtube-Data-CE')
ws = worksheet.worksheet("Track IDs")
rows = ws.get_all_values()
d_list = df1.to_numpy().tolist()
headers = df1.columns.tolist()
out_data = [headers] + d_list
ws.update('A34273',[headers] + d_list)

{'spreadsheetId': '1Iaz-jQr80ue8Jz5LQqIzqDgSTtYFSqrhZ_9fsWoOVNs',
 'updatedRange': "'Track IDs'!A34273:B35697",
 'updatedRows': 1425,
 'updatedColumns': 2,
 'updatedCells': 2850}

In [ ]:
worksheet = gc.open('Youtube-Data-CE')
ws = worksheet.worksheet("Track IDs")
rows = ws.get_all_values()
df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

In [ ]:
#spotify
def get_stats_data_spotify(track_id, refresh_token):
    stats_result = {
        'track_id': [],
        'spotify_popularity_streams': [],
        'timestp': []
    }

    for id in track_id:
        headers = {'Authorization': 'Bearer ' + refresh_token}
        stats_url = 'https://api.chartmetric.com/api/track/'+id+'/spotify/stats/most-history?since=2023-03-01'
        print(id, end=" ")
        stats_res = requests.get(stats_url, headers=headers)
        stats_data = stats_res.json().get('obj', [])

        if stats_data:
            for data in stats_data:
                for item in data.get('data', []):
                    stats_result['track_id'].append(id)
                    stats_result['spotify_popularity_streams'].append(item.get('value'))
                    stats_result['timestp'].append(item.get('timestp'))

        time.sleep(1)

    return stats_result

In [ ]:
refresh_token = get_refresh_token()
df1 = pd.DataFrame(get_stats_data_spotify(list(df['track_id'].iloc[6501:8501]), refresh_token))

19973976 20192931 20233531 20657008 22122353 22262000 22399997 22564241 67996606 67998644 69191349 73304849 79405854 79407367 89520864 89522053 92311678 94226166 20207724 20465043 20606930 21852999 22000401 22044582 22360351 22369251 22405749 22458736 13829033 13860618 15184156 15853751 16219253 16752367 16810912 17014020 17015466 17052211 11050083 11052594 11052595 11052596 11205489 11205490 11205491 11205492 11205498 11205500 11148933 11148934 11148935 13768884 13823636 13823639 13823645 13823648 13823649 13823650 25001647 25001648 25001649 25001650 25001713 25001714 25001715 25001834 25001870 25001871 19206558 19943958 20424431 20606780 20606895 20622727 22116467 22237860 22278461 22278462 11246606 11889622 11935130 11997836 13133585 13410162 13410237 13410321 13413368 13413372 13211505 13215686 13218824 13218825 13219558 13221750 13222055 13222153 13965058 13965059 11007795 11007796 12365533 12938861 13054342 13056941 13829893 14932365 15100461 15100462 13964787 13964888 13965071 1

In [ ]:
worksheet = gc.open('Youtube-Data-CE')
ws = worksheet.worksheet("Track Stats")
rows = ws.get_all_values()
d_list = df1.to_numpy().tolist()
headers = df1.columns.tolist()
out_data = [headers] + d_list
ws.update('A2',[headers] + d_list)

APIError: ignored

In [ ]:
#tiktok
def get_stats_data_tiktok(track_id, refresh_token):
    stats_result = {
        'track_id': [],
        'tiktok_posts_count': [],
        'timestp': []
    }

    for id in track_id:
        headers = {'Authorization': 'Bearer ' + refresh_token}
        stats_url = 'https://api.chartmetric.com/api/track/'+id+'/tiktok/stats/most-history?since=2023-03-01'
        print(id, end=" ")
        stats_res = requests.get(stats_url, headers=headers)
        stats_data = stats_res.json().get('obj', [])

        if stats_data:
            for data in stats_data:
                for item in data.get('data', []):
                    stats_result['track_id'].append(id)
                    stats_result['tiktok_posts_count'].append(item.get('value'))
                    stats_result['timestp'].append(item.get('timestp'))

        time.sleep(1)

    return stats_result

In [ ]:
refresh_token = get_refresh_token()
df2 = pd.DataFrame(get_stats_data_tiktok(list(df['track_id'].iloc[5051:6051]), refresh_token))

12447561 12451507 12451508 12451509 12451510 31385442 31411778 31480158 31538475 33432651 40933782 41385030 41527155 46574244 46823557 20431294 60766191 12112842 12288803 12290533 12292205 13030948 13030949 13030950 13030951 13030952 13030953 11123649 12182028 12182029 12182030 12182031 12182032 12182033 12182034 12182035 12182036 12450149 13074122 13074225 13074415 13074417 13074418 13074419 13076210 13077250 13077461 10980446 11503395 11557024 11659911 11752310 12463392 12499667 12546063 13100559 13961291 11168431 11170113 11189590 11309804 12050318 12332891 12984506 14614473 15464292 15464293 11009751 11009752 11009753 11009754 11009755 11009762 11009763 11009764 11009765 11009766 12064483 13030743 13030746 13030747 13030840 13030870 13030996 13031044 13031086 13031087 94034816 99990536 100272591 103974052 104646886 11010548 11010590 14348873 14348874 14353161 14369044 14498466 14498467 14498477 14498479 11291431 11291472 11291473 11291474 11291475 11291476 11291533 11291619 1129165

In [ ]:
worksheet = gc.open('Youtube-Data-CE')
ws = worksheet.worksheet("Track Stats")
rows = ws.get_all_values()
d_list = df2.to_numpy().tolist()
headers = df2.columns.tolist()
out_data = [headers] + d_list
ws.update('E50099',[headers] + d_list)

{'spreadsheetId': '1Iaz-jQr80ue8Jz5LQqIzqDgSTtYFSqrhZ_9fsWoOVNs',
 'updatedRange': 'Track_Stats!E49591:G50099',
 'updatedRows': 509,
 'updatedColumns': 3,
 'updatedCells': 1527}

In [ ]:
#youtube
def get_stats_data_youtube(track_id, refresh_token):
    stats_result = {
        'track_id': [],
        'youtube_views': [],
        'timestp': []
    }

    for id in track_id:
        headers = {'Authorization': 'Bearer ' + refresh_token}
        stats_url = 'https://api.chartmetric.com/api/track/'+id+'/youtube/stats/most-history?since=2023-03-01'
        print(id, end=" ")
        stats_res = requests.get(stats_url, headers=headers)
        stats_data = stats_res.json().get('obj', [])

        if stats_data:
            for data in stats_data:
                for item in data.get('data', []):
                    stats_result['track_id'].append(id)
                    stats_result['youtube_views'].append(item.get('value'))
                    stats_result['timestp'].append(item.get('timestp'))

        time.sleep(1)

    return stats_result

In [ ]:
refresh_token = get_refresh_token()
df3 = pd.DataFrame(get_stats_data_youtube(list(df['track_id'].iloc[2001:3001]), refresh_token))

70976227 75979639 80260460 82087549 82454270 82529068 83376775 84192516 84335991 84347166 12194299 12364193 12364194 12364195 12364196 12364197 12364198 12364199 12364200 12364201 80861198 80876338 80876339 80876340 80876349 90326075 94199822 94201765 94201766 94202168 13396722 13396723 13396724 13396725 20378237 29510009 29511484 29511485 29511486 29511487 13391052 13391061 13391062 13391063 13391064 13391065 13391114 13391216 13391217 13391218 12376383 12376388 12376400 12376402 12854890 12854978 12854990 12854992 12854995 12855000 15086302 15086386 15086387 15086416 15086429 15086430 15086431 15086432 15086433 15086434 13383996 13383997 13383998 13383999 13384000 14664748 14664749 14664750 14664751 14664752 12493848 12494047 12494148 12494149 12494150 12494156 12494182 12494326 13919570 13921791 12310143 12619757 12619874 12813349 12813422 12813428 12813429 12813430 12813432 12813488 10990208 10990480 11651161 11651163 12257265 12257267 12257270 12310114 12334210 12334211 31061637 3

In [ ]:
worksheet = gc.open('Youtube-Data-CE')
ws = worksheet.worksheet("Track Stats")
rows = ws.get_all_values()
d_list = df3.to_numpy().tolist()
headers = df3.columns.tolist()
out_data = [headers] + d_list
ws.update('I99537',[headers] + d_list)

APIError: ignored

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheet = gc.open('Youtube-Data-CE')
ws = worksheet.worksheet("Track IDs")
rows = ws.get_all_values()

track_df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

In [ ]:
# Fetching Metadata:
def get_track_data(track_id, refresh_token):
    stats_result = {'Track_Id': [],
                    'Track_Name': [],
                    'Album_Release_Date': [],
                    'Popularity_Rating': [],
                    'Num_Spotify_Playlists': [],
                    'Num_YouTube_Playlists': [],
                    'Spotify_Playlist_Total_Reach': [],
                    'YouTube_Playlist_Total_Reach': [],
                    'Spotify_Popularity':[],
                    'Num_TikTok_Videos': [],
                    'TikTok_Counts': [],
                    'Spotify_Streams': [],
                    'YouTube_Views': [],
                    'Track_Duration': []}

    for id in track_id:
      headers= {'Authorization' : 'Bearer '+refresh_token}
      stats_url = 'https://api.chartmetric.com/api/track/'+str(id)
      stats_res = requests.get(stats_url, headers=headers)
      print(id, end=" ")
      stats_result['Track_Id'].append(id)
      if 'name' not in stats_res.json()['obj']:
        stats_result['Track_Name'].append('Not Available')
      else:
        stats_result['Track_Name'].append(stats_res.json()['obj']['name'])
      stats_result['Album_Release_Date'].append(stats_res.json()['obj']['release_date'])
      if len(stats_res.json()['obj']['albums']) == 0:
        stats_result['Popularity_Rating'].append(0)
      else:
        stats_result['Popularity_Rating'].append(stats_res.json()['obj']['albums'][0]['popularity'])
      stats_result['Num_Spotify_Playlists'].append(stats_res.json()['obj']['cm_statistics']['num_sp_playlists'])
      stats_result['Num_YouTube_Playlists'].append(stats_res.json()['obj']['cm_statistics']['num_yt_playlists'])
      stats_result['Spotify_Playlist_Total_Reach'].append(stats_res.json()['obj']['cm_statistics']['sp_playlist_total_reach'])
      stats_result['YouTube_Playlist_Total_Reach'].append(stats_res.json()['obj']['cm_statistics']['yt_playlist_total_reach'])
      stats_result['Spotify_Popularity'].append(stats_res.json()['obj']['cm_statistics']['sp_popularity'])
      stats_result['Num_TikTok_Videos'].append(stats_res.json()['obj']['cm_statistics']['num_tt_videos'])
      stats_result['TikTok_Counts'].append(stats_res.json()['obj']['cm_statistics']['tiktok_counts'])
      stats_result['Spotify_Streams'].append(stats_res.json()['obj']['cm_statistics']['sp_streams'])
      stats_result['YouTube_Views'].append(stats_res.json()['obj']['cm_statistics']['youtube_views'])
      if 'duration_ms' not in stats_res.json()['obj']:
        stats_result['Track_Duration'].append(0)
      else:
        stats_result['Track_Duration'].append(stats_res.json()['obj']['duration_ms'])
      time.sleep(1)
    return stats_result

In [ ]:
result = pd.DataFrame(get_track_data(list(track_df['track_id'].iloc[5502:6001]), refresh_token))

In [ ]:
result = result.fillna(0)

worksheet = gc.open('Youtube-Data-CE')
ws = worksheet.worksheet("Track Metadata")
rows = ws.get_all_values()
d_list = result.to_numpy().tolist()
headers = result.columns.tolist()
out_data = [headers] + d_list
ws.update('A5297',[headers] + d_list)